In [17]:
import torch
import numpy as np
from collections import defaultdict
from scipy.stats import spearmanr, pearsonr
from scipy.stats.stats import kendalltau as kendallr

In [105]:
np.array(range(64)).reshape(4,16).reshape(8,8)

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
       [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31],
       [32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47],
       [48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]])

In [101]:
k

[3, 2, 1, 4]

In [94]:
results = torch.load('results/results_lsvq_s32*32_ens6.pkl')

In [91]:
def rescale(pr, gt=None):
    if gt is None:
        pr = ((pr - np.mean(pr)) / np.std(pr))
    else:
        pr = ((pr - np.mean(pr)) / np.std(pr)) * np.std(gt) + np.mean(gt)
    return pr

all_datasets = ['LIVE_VQC', 'KoNViD', 'CVD2014', 'LSVQ']

def pyramid_ensemble_coefficients(results):
    ens_result_len = results[0].get('pr_labels').shape[0] // 4
    
    gt_labels = [r['gt_label'] for r in results]
    
    p_results = defaultdict()
    
    for i in range(1, ens_result_len + 1):
        scores = np.zeros(4)
        for j in range(ens_result_len - i + 1):
            pr_labels = [np.mean(r['pr_labels'][j*4:(j+i)*4]) for r in results]
            pr_labels = rescale(pr_labels, gt_labels)
            scores += np.array([spearmanr(gt_labels, pr_labels)[0], pearsonr(gt_labels, pr_labels)[0], kendallr(gt_labels, pr_labels)[0], np.sqrt(((gt_labels - pr_labels) ** 2).mean())])
        scores /= ens_result_len - i + 1
        p_results[i] = scores
        
    return p_results

def pyramid_ensemble_stds(results):
    ens_result_len = results[0].get('pr_labels').shape[0] // 4
    
    gt_labels = [r['gt_label'] for r in results]
    
    s_results = defaultdict()
    
    for i in range(1, ens_result_len + 1):
        all_pr_labels = []
        stds = np.zeros(2)
    
        for j in range(ens_result_len - i + 1):
            pr_labels = [np.mean(r['pr_labels'][j*4:(j+i)*4]) for r in results]
            pr_labels = rescale(pr_labels, gt_labels)
            all_pr_labels.append(pr_labels)

        all_pr_labels = np.stack(all_pr_labels, 1)
        for q in range(2):
            stds[q] = np.mean(np.std(all_pr_labels, q))
        s_results[i] = stds
    return s_results

In [97]:
pyramid_ensemble_coefficients(results[:7186])

defaultdict(None,
            {1: array([0.87262503, 0.87225169, 0.68969995, 5.31474978]),
             2: array([0.87391015, 0.87343881, 0.6912761 , 5.28999947]),
             3: array([0.87441983, 0.87385815, 0.69189289, 5.28122945]),
             4: array([0.87463606, 0.87405689, 0.69217673, 5.27706804]),
             5: array([0.87471011, 0.87415792, 0.69229663, 5.27495101]),
             6: array([0.87479845, 0.87423328, 0.69246272, 5.27337138])})

In [98]:
pyramid_ensemble_stds(results[:7186])

defaultdict(None,
            {1: array([10.51454163,  0.61519825]),
             2: array([10.51454353,  0.37080199]),
             3: array([10.51454258,  0.23773439]),
             4: array([10.51454639,  0.14751394]),
             5: array([10.51453972,  0.07975236]),
             6: array([10.51454544,  0.        ])})